In [ ]:
from cleanfid import fid
fdir1 = 'retrieve/retrieve_dog/photo_of_a_dog'
fdir2 = 'logs/2023-03-26T12-52-38_dog-sdv4/samples'
score = fid.compute_kid(fdir1, fdir2)
score

# Compute new_concept_similarity


In [ ]:
from sklearn.manifold import TSNE
import numpy as np
from matplotlib import pyplot as plt
import torch

def to_array(dl):
    if isinstance(dl[0],torch.Tensor):
        out = [d.cpu().numpy() for d in dl]
        return np.stack(out, axis=0)
    elif isinstance(dl[0],np.ndarray):
        return np.stack(dl, axis=0)
    else:
        raise NotImplementedError('not supported type')

def process_data(path):
    out={}
    data = torch.load(path)
    c_sim = to_array(data.pop('c_sim_global'))
    c_new_global =  to_array(data.pop('c_new_global'))
    c_reg_global = to_array(data.pop('c_reg_global'))
    keys = data.keys()
    out['c_sim'] = c_sim
    # out['c_sim'] = c_sim
    for key in keys:
        weights = to_array(data[key])
        start_weight =  weights[0]
        delta_weights = weights - np.expand_dims(start_weight, axis=0)       
        # distance = np.sum(delta_weights,axis=(1,2))
        distance = np.linalg.norm(delta_weights,axis=(1,2))
        out[key + '_distance'] = distance
        # print(distance)
        # print(np.transpose(weights[1],(1,0)).shape)
        c_mapped_new = [c_n @ np.transpose(weights[i],(1,0)) for i,c_n in zip(range(len(weights)),c_new_global)]
        c_mapped_new = np.stack(c_mapped_new, axis=0)
        c_mapped_reg = [c_reg_global @ np.transpose(weights[i],(1,0)) for i in range(len(weights))]
        c_mapped_reg = np.concatenate(c_mapped_reg, axis=0)
        # print(c_mapped_reg.shape)
        out[key + '_c_mapped_new'] = c_mapped_new
        out[key + '_c_mapped_reg'] = c_mapped_reg
    return out



def draw_line(x1,x2 =None,x3= None,x4=None):
    fig, ax = plt.subplots()
    ax.plot(x1, marker='o', linestyle='-', color='r',label = '<new> + prompt')
    if x2 is not None:
        ax.plot(x2, marker='o', linestyle='--', color='g',label = '<new> + prompt, reg')
    if x3 is not None:
        ax.plot(x3, marker='s', linestyle='-', color='r',label = '<new>')
    if x4 is not None:
        ax.plot(x4, marker='s', linestyle='--', color='g',label = '<new>, reg')
    plt.legend( loc='upper right',fontsize=7)
    plt.xticks(np.arange(0,len(x1) +1),fontsize=7)
    plt.show()

path = 'logs/2023-03-27T09-11-12_dog_realReg-sdv4/statics.pt'
path_reg = 'logs/2023-03-27T09-10-44_dog_genReg-sdv4/statics.pt'
out = process_data(path)
out_reg = process_data(path_reg)
c_mapped_new = []
c_mapped_reg = []
distance = []
for key in out_reg.keys():
    if 'distance' in key:
        distance.append(key)
    elif 'c_mapped_new' in key:
        c_mapped_new.append(key)
    elif 'c_mapped_reg' in key:
        c_mapped_reg.append(key)
c_mapped_reg = sorted(c_mapped_reg)
c_mapped_new = sorted(c_mapped_new)
distance = sorted(distance) 


N = len(distance)
n = 4 # size of each array

fig, axs = plt.subplots(N // 4 + (N % 4 != 0), 4,figsize=(10, 8))
for i in range(N):
    data = out[distance[i]]
    data_reg = out_reg[distance[i]]
    data_c_reg = out_c_reg[distance[i]]
    axs[i // 4][i % 4].plot(data, marker='o', linestyle='-', color='r',linewidth=1,label = 'real reg.')
    axs[i // 4][i % 4].plot(data_reg, marker='o', linestyle='-', color='b',linewidth=1,label = 'gen. reg.')
    axs[i // 4][i % 4].plot(data_c_reg, marker='o', linestyle='-', color='g',linewidth=1,label = 'gen. reg.')
    # plt.xticks(fontsize=3)
    # plt.yticks(fontsize=3)
plt.savefig('distance.png',dpi=300)
plt.show()

我给你一个句子，需要你帮助我生成100句语义相近的句子，生成的句子内容可以拓展和补充。以下是一个例子，我给你一句“photo of a dog", 你可以生成“photo of a corgi on the beach"，或“A yellow dog rides a skateboard on the street”，如果明白了需求，只需要回复“OK”

In [ ]:
import os,torch
import copy
path = 'logs/2023-04-03T16-00-34_dogreg_1_scale0.01_a_dog/checkpoints/'
ckpts = []
for name in os.listdir(path):
    path_file = os.path.join(path,name)
    ckpt = dict(torch.load(path_file, map_location='cpu')['state_dict'])
    keys = ckpt.keys()
    for k in list(keys):
        if ('attn2.to_k' in k or 'attn2.to_k0' in k or 'attn2.to_v0' in k or 'attn2.to_v' in k):
            pass 
        else:
            ckpt.pop(k)
    ckpts.append(ckpt)
epoch_ckpt = ckpts[-1]

delta = epoch_ckpt['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn2.to_k.weight'] - epoch_ckpt['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn2.to_k0.weight'] 
flatten_delta2 = delta.flatten()
import matplotlib.pyplot as plt
fig, ax= plt.subplots(figsize=(6,6))
ax.hist(flatten_delta,bins=100,label=r"$\alpha = 1$,$\beta$ = 1")
ax.hist(flatten_delta2,bins=100,label=r"$\alpha$ = 10,$\beta$ = 1")
plt.legend()
plt.show()

SVD compression

In [ ]:
W = epoch_ckpt['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn2.to_k.weight']
W0 = torch.load('Stable-diffusion/sd-v1-4-full-ema.ckpt',map_location='cpu')['state_dict']['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn2.to_k.weight']
delta = W - W0
u, s, vt = torch.linalg.svd(delta.clone())

# Get_deltas

In [ ]:
import glob
import os
for dir in os.listdir('logs'):
    path = os.path.join('logs',dir)
    os.system(f'python src2/get_deltas.py --path {path}')

In [1]:
from src2.custom_modules import FrozenCLIPEmbedderWrapper as CLIP2
from src.custom_modules import FrozenCLIPEmbedderWrapper as CLIP1
import torch
clip1 = CLIP1("<new1>",device='cpu')
clip2 = CLIP2("<new1>",device='cpu')


/data/zhicai/miniconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.9.self_attn.out_proj.weight', 'vision_model.encoder.layers.13.layer_norm1.weight', 'vision_model.encoder.layers.22.self_attn.v_proj.weight', 'vision_model.encoder.layers.12.self_attn.v_proj.bias', 'vision_model.encoder.layers.2.layer_norm2.bias', 'vision_model.encoder.layers.10.layer_norm1.bias', 'vision_model.encoder.layers.0.self_attn.k_proj.bias', 'vision_model.encoder.layers.15.self_attn.out_proj.bias', 'vision_model.encoder.layers.20.mlp.fc1.bias', 'vision_model.encoder.layers.3.mlp.fc2.weight', 'vision_model.encoder.layers.2.mlp.fc2.bias', 'vision_model.encoder.layers.10.mlp

In [27]:
a1

tensor([[[-0.3884,  0.0229, -0.0522,  ..., -0.4899, -0.3066,  0.0675],
         [ 0.0290, -1.3258,  0.3085,  ..., -0.5257,  0.9768,  0.6652],
         [ 1.1565,  0.1318,  0.7895,  ..., -2.1024, -1.1519, -0.3311],
         ...,
         [ 0.2664,  0.0406, -0.1670,  ...,  0.7111, -0.0082, -0.1973],
         [ 0.2704,  0.0392, -0.1608,  ...,  0.7331, -0.0099, -0.2198],
         [ 0.2096,  0.1045, -0.1389,  ...,  0.7044, -0.0060, -0.1833]]],
       grad_fn=<NativeLayerNormBackward0>)

In [32]:
b1

tensor([[[-3.4725e-01,  2.0514e-02, -4.6669e-02,  ..., -4.3801e-01,
          -2.7413e-01,  6.0311e-02],
         [ 2.5958e-02, -1.1854e+00,  2.7579e-01,  ..., -4.7001e-01,
           8.7337e-01,  5.9473e-01],
         [ 1.0340e+01,  1.1788e+00,  7.0588e+00,  ..., -1.8798e+01,
          -1.0299e+01, -2.9606e+00],
         ...,
         [ 2.3819e-01,  3.6322e-02, -1.4935e-01,  ...,  6.3581e-01,
          -7.3146e-03, -1.7644e-01],
         [ 2.4178e-01,  3.5090e-02, -1.4381e-01,  ...,  6.5546e-01,
          -8.8145e-03, -1.9650e-01],
         [ 1.8737e-01,  9.3452e-02, -1.2416e-01,  ...,  6.2979e-01,
          -5.3538e-03, -1.6390e-01]]], grad_fn=<MulBackward0>)

In [ ]:
import glob
import numpy as np
import re
dir = 'logs/logs_liox/2023-04-06T15-39-11_tortoise_plushy-sdv4'
matchObj = re.match(r'.*(tortoise_plushy|teddybear|cat|dog).*', dir, re.M|re.I)
concept = matchObj[1].replace('_',' ')
print(concept)
sims = []
for delta_ckpt in sorted(glob.glob(dir+'/checkpoints/*.ckpt')):
    if delta_ckpt is not None:
        delta_st = torch.load(delta_ckpt,map_location='cpu')
        embed = None
        if 'embed' in delta_st['state_dict']:
            embed = delta_st['state_dict']['embed'].reshape(-1,768)
            del delta_st['state_dict']['embed']
        clip.transformer.text_model.embeddings.token_embedding.weight.data[-embed.shape[0]:] = embed
    a = clip.encode_text(f'<new1> {concept}')
    b = clip.encode_text(f'{concept}')
    sim = torch.cosine_similarity(a[1],b[1])
    sims.append(np.round(sim.item(),4))
print(sims)

In [ ]:
cat_0_0 = [0.7059, 0.7015, 0.7048, 0.7049, 0.7062]
cat_01_0 = [0.6662, 0.6095, 0.5932, 0.5752, 0.5783]
cat_10_0 = [0.6705, 0.5937, 0.5625, 0.524, 0.4937]
sim_cat = [0.7011, 0.6720]

bear_0_0 = [0.7568, 0.7575, 0.757, 0.7559, 0.7567]
bear_01_0 = [0.7401, 0.6886, 0.664, 0.6285, 0.6267]
bear_01_0 = [0.7377, 0.673, 0.6286, 0.5637, 0.5556]
sim_teddybear = [0.6466, 0.6668]

dog_0_0 = [0.6241, 0.6054, 0.6097, 0.608, 0.6343]
dog_01_0 = [0.5841, 0.5235, 0.5014, 0.51, 0.5088]
dog_01_0_onlyK = [0.6115, 0.5835, 0.5924, 0.5809]
sim_dog = [0.5982, 0.6258]

plushy_01_0_onlyK = [0.807, 0.7913, 0.7771, 0.7607, 0.7532]
plushy_01_0_onlyV = [0.8067, 0.79, 0.7706, 0.754, 0.7436]
sim_plushy = [0.7917, 0.802]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def draw_line2(x1,x2 =None,x3= None,x4=None,constant1=None,constant2=None):
    fig, ax = plt.subplots()
    ax.plot(x1, marker='*', linestyle='-', color='r',label = 'no reg.')
    if x2 is not None:
        ax.plot(x2, marker='o', linestyle='-', color='g',label = '0.1 text reg.')
    if x3 is not None:
        ax.plot(x3, marker='s', linestyle='-', color='b',label = '1.0 text reg.')
    if x4 is not None:
        ax.plot(x4, marker='s', linestyle='--', color='g',label = '<new>, reg')
    if constant1 is not None:
        # draw a horizontal line
        ax.axhline(y=constant1, color='r', linestyle='--',label = 'real reg.')
    if constant2 is not None:
        # draw a horizontal line
        ax.axhline(y=constant2, color='y', linestyle='--',label = 'gen. reg.')
        
    plt.legend( loc='best',fontsize=10)
    plt.xticks(np.arange(0,len(x1) +1),fontsize=7)
    plt.show()
draw_line2(cat_0_0,cat_01_0,cat_10_0,constant1 = sim_cat[0],constant2 = sim_cat[1])

In [ ]:
sim_dog_epoch = [0.5962,0.6115,0.5835,0.5924,0.5809]
sim_dog = [0.5982, 0.6258]

sim_tortoise_plushy_epoch = [0.8040, 0.7401,0.6886,0.6640,0.6285,0.6267]

sim_cat_epoch = [0.7061, 0.7401,0.6886,0.6640,0.6285,0.6267]
sim_cat = [0.7011, 0.6720, 0.6267]

sim_teddybear_epoch = [0.7455, 0.7401,0.6886,0.6640,0.6285,0.6267]
sim_teddybear = [0.7611, 0.7369, 0.6267]

In [ ]:
import re

str = "2023-04-05T10-16-22_tortoise_plushyreg_1_scale1_a_tortoise_plushy_ridge"
matchObj = re.match(r'.*_(.*)reg.*a_(.*)_(.*)', str, re.M|re.I)

if matchObj:
   print("matchObj.group(1) : ", matchObj.group(1))
   print("matchObj.group(2) : ", matchObj.group(2))
   print("matchObj.group(3) : ", matchObj.group(3))
else:
   print("No match!!")

file = 'logs/2023-04-05T15-42-14_catreg_0.1_scale0.1_a_cat_ridge/checkpoints/epoch=000011-step=000000399.ckpt'
out = re.match(r'.*step=0+(\d+).ckpt',file)

In [ ]:
from PIL import Image
import os

dirs = ['logs/2023-04-07T21-44-17_tortoise_plushyreg_0.1_scale0_a_tortoise_plushy_ridge_onlyK', 'logs/2023-04-08T03-32-12_tortoise_plushyreg_0.1_scale0_a_tortoise_plushy_ridge_onlyK_noblip', 'logs/logs_toTest/2023-04-09T09-30-52_tortoise_plushyreg_0-0_scale0-0_a_tortoise_plushy', 'logs/logs_liox/2023-04-05T14-20-45_tortoise_plushy-sdv4','logs/logs_liox/2023-04-06T15-39-11_tortoise_plushy-sdv4']
profix = ['delta_epoch=000016-step','epoch=000016-step=000000249','last','delta_epoch=last','delta_epoch=last']
output_dir = 'output/tortoise_plushyreg/'
os.makedirs(output_dir, exist_ok=True)
for filename in os.listdir(dirs[0]):
    if filename.endswith('.png') and filename.startswith(profix[0]):
        filename = filename.replace(profix[0],'')
        
        images = [Image.open(d + '/' + ''.join([p,filename])) for d,p in zip(dirs,profix)]
        widths, heights = zip(*(i.size for i in images))
        # total_width = max(widths)
        total_width = widths[0]
        max_height = len(dirs) * heights[0]
        new_im = Image.new('RGB', (total_width, max_height))
        x_offset = 0
        for im in images:
            new_im.paste(im, (0, x_offset))
            x_offset += min(im.size[1],heights[0])
        new_im.save(os.path.join(output_dir, filename))
        print("saved",filename)

In [ ]:
dirs = ['logs/2023-04-07T18-11-29_catreg_0.1_scale0_a_cat_ridge_onlyK', 'logs/2023-04-08T01-55-33_catreg_0.1_scale0_a_cat_ridge_onlyK_noblip', 'logs/2023-04-08T04-29-59_catreg_0_scale0_a_cat_ridge_onlyK_noblip', 'logs/logs_liox/2023-04-05T05-04-21_cat-sdv4','logs/logs_liox/2023-04-06T15-07-33_cat-sdv4']
profix = ['delta_epoch=000035-step','delta_epoch=000035-step','epoch=000035-step=000000249','delta_epoch=last','delta_epoch=last']
output_dir = 'output/cat_reg/'

os.makedirs(output_dir, exist_ok=True)
for filename in os.listdir(dirs[0]):
    if filename.endswith('.png') and filename.startswith(profix[0]):
        filename = filename.replace(profix[0],'')
        try:
            images = [Image.open(d + '/' + ''.join([p,filename])) for d,p in zip(dirs,profix)]
        except:
            print("error",filename)
            continue
        widths, heights = zip(*(i.size for i in images))
        total_width = max(widths)
        max_height = sum(heights)
        new_im = Image.new('RGB', (total_width, max_height))
        x_offset = 0
        for im in images:
            new_im.paste(im, (0, x_offset))
            x_offset += im.size[1]
        new_im.save(os.path.join(output_dir, filename))
        print("saved",filename)

In [ ]:
dirs = ['logs/2023-04-07T20-04-06_teddybearreg_0.1_scale0_a_teddybear_ridge_onlyK', 'logs/2023-04-08T02-52-02_teddybearreg_0.1_scale0_a_teddybear_ridge_onlyK_noblip', 'logs/2023-04-08T04-49-07_teddybearreg_0_scale0_a_teddybear_ridge_onlyK_noblip', 'logs/logs_liox/2023-04-05T05-15-20_teddybear-sdv4','logs/logs_liox/2023-04-06T15-24-25_teddybear-sdv4']
profix = ['delta_epoch=000027-step','epoch=000027-step=000000249','epoch=000027-step=000000249','delta_epoch=last','delta_epoch=last']
output_dir = 'output/teddybear_reg/'

os.makedirs(output_dir, exist_ok=True)
for filename in os.listdir(dirs[0]):
    if filename.endswith('.png') and filename.startswith(profix[0]):
        filename = filename.replace(profix[0],'')
        
        images = [Image.open(d + '/' + ''.join([p,filename])) for d,p in zip(dirs,profix)]
        widths, heights = zip(*(i.size for i in images))
        # total_width = max(widths)
        total_width = widths[0]
        max_height = sum(heights)/5 * 4
        new_im = Image.new('RGB', (total_width, max_height))
        x_offset = 0
        for im in images:
            new_im.paste(im, (0, x_offset))
            x_offset += min(im.size[1],heights[0])
        new_im.save(os.path.join(output_dir, filename))
        print("saved",filename)

In [ ]:

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# Set up the grid
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
x, y = np.meshgrid(x, y)

# Define the horizontal plane
z1 = np.zeros_like(x)
z1.fill(1.1)

# Define the Gaussian plane
z2 = np.exp(-(x**2 + y**2))+0.9

# Calculate the volume under the Gaussian plane
# vol_z2 = np.trapz(np.trapz(z2, x[0,:]), y[:,0])

# Adjust the height of the horizontal plane to match the volume under the Gaussian plane
# z1 = z1 + vol_z2 / (x.shape[0] * y.shape[1])

# Plot the planes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x, y, z1, alpha =0.5)
ax.plot_surface(x, y, z2, alpha =0.5)

plt.show()